In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# 필요한 모듈을 설치합니다
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch #원래 ==1.3.1

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x3a20c000 @  0x7fa86477a615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 15kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 9.1MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588532 sha256=83a28ac4f66138a176c0f2307e6e2f9fe8978ea8e29232cbed80732bbe5a3159
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 12.1M

In [ ]:
#SKT에서 공개한 KoBERT 모델을 불러옵니다 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-hn40515x
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-hn40515x
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=65e78cb71121407f4f3123e9730f6e75e4789fd45ca15e9ef7e1b1ff55ef3de3
  Stored in directory: /tmp/pip-ephem-wheel-cache-7mzdgl5k/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import WarmupLinearSchedule 

## build model

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0") if cuda.is_available() else torch.device("cpu")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5

## load data

In [ ]:
df = pd.read_csv("news_data/news_train.csv")
dtls = []
for (content, info) in zip(df["content"], df["info"]):
  dtls.append([content, info])

dtls[:10]

[['[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도', 0],
 ['"실적기반" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개', 1],
 ['하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시', 1],
 ['종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지', 0],
 ['전국적인 소비 붐 조성에 기여할 예정', 0],
 ['[이데일리 권오석 기자] 중소벤처기업부(이하 중기부)는 대한민국 동행세일에 7개 TV홈쇼핑사가 홍보와 판매에 동참한다고 26일 밝혔다',
  0],
 ['대한민국 동행세일은 라이브 커머스, 언택트 콘서트, O2O 행사 연계 등 비대면이라는 새로운 형태의 소비촉진 행사다', 0],
 ['6개 권역에서의 현장행사와 온·오프라인 판촉, TV홈쇼핑 등 연계행사를 통해 소비심리를 진작하기 위함이다', 0],
 ['이번 동행세일에서는 롯데·공영·CJ·현대·GS·NS·홈앤쇼핑 등 7개 TV 홈쇼핑사가 동행세일의 홍보와 방송 판매에 동참해 전국적인 소비 붐 조성에 기여한다',
  0],
 ['동행세일 기간 중 홈쇼핑사에서는 판매방송 사이에 영상을 노출하는 방식(SB, station break)을 통해 행사 취지와 메시지를 홍보한다',
  0]]

In [ ]:
# KorEDA

import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)

########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, num_aug=2):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

In [ ]:
# train, validation, test set을 나눠주세요
from sklearn.model_selection import train_test_split

dataset_train, dataset_eval = train_test_split(dtls, test_size=0.1, random_state=123)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_eval = BERTDataset(dataset_eval, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
eval_dataloader = torch.utils.data.DataLoader(data_eval, batch_size=batch_size, num_workers=5)

In [ ]:
#모델을 만들고 GPU 사용 설정을 해줍니다
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
#옵티마이저와 손실함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

#정확도를 계산하기 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#학습 과정

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad() 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        label = label.long().to(device) 
        out = model(token_ids, valid_length, segment_ids) 
        loss = loss_fn(out, label) 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step() 
        scheduler.step()  # Update learning rate schedule 
        train_acc += calc_accuracy(out, label)  
        if batch_id % log_interval == 0: 
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1))) 
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1))) 
    
    model.eval() #모델 평가 부분 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(eval_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

    if test_acc > best_acc:
      best_acc = test_acc
      best_model_wts = copy.deepcopy(model.state_dict())
    

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 1 batch id 1 loss 0.7124730944633484 train acc 0.609375
epoch 1 batch id 201 loss 0.1435549259185791 train acc 0.9079601990049752
epoch 1 batch id 401 loss 0.24954089522361755 train acc 0.9416302992518704
epoch 1 batch id 601 loss 0.12826129794120789 train acc 0.9541129367720466
epoch 1 batch id 801 loss 0.061839669942855835 train acc 0.9614934456928839
epoch 1 batch id 1001 loss 0.01616845093667507 train acc 0.966314935064935
epoch 1 batch id 1201 loss 0.007347277831286192 train acc 0.969738759367194
epoch 1 batch id 1401 loss 0.08873249590396881 train acc 0.9723300321199143
epoch 1 batch id 1601 loss 0.015013745985925198 train acc 0.97446908182386

epoch 1 train acc 0.9751029191616767



epoch 1 test acc 0.991179435483871


epoch 2 batch id 1 loss 0.035869769752025604 train acc 0.984375
epoch 2 batch id 201 loss 0.004452013410627842 train acc 0.9910603233830846
epoch 2 batch id 401 loss 0.05022222921252251 train acc 0.991427680798005
epoch 2 batch id 601 loss 0.005578753538429737 train acc 0.9921745008319468
epoch 2 batch id 801 loss 0.00037079129833728075 train acc 0.9929190074906367
epoch 2 batch id 1001 loss 0.02063818834722042 train acc 0.9932723526473527
epoch 2 batch id 1201 loss 0.002275320002809167 train acc 0.9937552039966694
epoch 2 batch id 1401 loss 0.05700036510825157 train acc 0.9940332798001428
epoch 2 batch id 1601 loss 0.008724571205675602 train acc 0.9940857276702061

epoch 2 train acc 0.9941523203592815



epoch 2 test acc 0.9948756720430108


epoch 3 batch id 1 loss 0.08656598627567291 train acc 0.984375
epoch 3 batch id 201 loss 0.010456174612045288 train acc 0.9952580845771144
epoch 3 batch id 401 loss 0.0006034760735929012 train acc 0.9964152119700748
epoch 3 batch id 601 loss 0.0003829226188827306 train acc 0.9968801996672213
epoch 3 batch id 801 loss 0.00012086035712854937 train acc 0.997249531835206
epoch 3 batch id 1001 loss 0.000393789290683344 train acc 0.9972839660339661
epoch 3 batch id 1201 loss 0.0002037075173575431 train acc 0.9975020815986678
epoch 3 batch id 1401 loss 0.0444573275744915 train acc 0.9976467701641685
epoch 3 batch id 1601 loss 0.007535972632467747 train acc 0.9976967520299813

epoch 3 train acc 0.9977544910179641



epoch 3 test acc 0.995127688172043


In [ ]:
torch.save(model.state_dict(), 'news_result/M_e3_b64.pt')
#torch.save(best_model_wts, 'news_result/M_augmentation_e5_b64_bestmodel.pt')

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load('news_result/M_e3_b64.pt'))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
import pandas as pd

df = pd.read_csv("news_data/news_test.csv")
test_dtls = []
for (id, content) in zip(df["id"], df["content"]):
  test_dtls.append([id, content, 99])

test_dtls[:10]

[['NEWS00237_1', "마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가'", 99],
 ['NEWS00237_2',
  '[이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로 발광다이오드(LED) 시대를 앞두고 주목받고 있습니다',
  99],
 ['NEWS00237_3',
  '최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가를 달리고 있습니다',
  99],
 ['NEWS00237_4',
  'TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서 수주가 더욱 늘어날 것이란 기대가 반영되는 모습입니다',
  99],
 ['NEWS00237_5',
  '코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에 상장했고, 창업주인 박명순 대표가 특수관계인과 함께 지분 50.76%를 보유하며 최대주주로 있습니다',
  99],
 ['NEWS00237_6',
  '사업부문은 반도체·레이저 장비 단일 사업부로 구성돼 있고 매출 유형은 반도체 제조용 장비, 레이저 응용 장비 등입니다',
  99],
 ['NEWS00237_7', '관련 업계에 따르면 이 회사는 최근 마이크로 LED TV 관련 장비의 양산에 돌입했습니다', 99],
 ['NEWS00237_8', '에어팟 등 무선이어폰용 레이저 컷팅 장비도 공급하기 시작했습니다', 99],
 ['NEWS00237_9',
  '코세스 측은 고객사와의 비밀 유지 조항이 있다며 구체적인 사항에 대해서는 알릴 수 없다는 입장입니다',
  99],
 ['NEWS00237_10',
  '하지만 코세스 주가는 지난달 21.4% 상승한 뒤 이달 들어서도 이미 16% 넘게 오르며 사업 확장에 대한 기대를 적극 반영하는 모습입니다',
  99]]

In [ ]:
data_test = BERTDataset(test_dtls, 1, 2, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
# 해당 데이터에 대해 분류를 시작합니다
model.eval()
answer=[]
train_acc = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  max_vals, max_indices = torch.max(out, 1)
  answer.append(max_indices.cpu().clone().numpy())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
from pandas.core.common import flatten

result = pd.DataFrame(df["id"], columns=["id"])
pred = pd.DataFrame(list(flatten(answer)), columns=["info"])
result = pd.concat([result, pred], axis=1)
result

,id,info
0,NEWS00237_1,1
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1


In [ ]:
result.to_csv("news_result/M_augmentation_e5_b64_bestmodel.csv", index=False)